<a href="https://colab.research.google.com/github/chirag-sharma-00/cs182-cv-project/blob/main/all_models_imagenetp_results.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torchvision
import matplotlib.pyplot as plt
import pandas as pd
import glob
import pathlib
import tqdm
import os
import time
import copy
from __future__ import print_function
from __future__ import division
from torchvision import datasets, models, transforms
from torch.utils.data import Dataset
from PIL import Image
from google.colab import drive
import shutil

In [2]:
!rm -rf sample_data

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

In [5]:
drive.mount('/content/drive', force_remount=True)

Mounted at /content/drive


## Load Datasets

In [ ]:
!unzip drive/MyDrive/182-cv-project/data/tiny-imagenet-200.zip
!tar -xvf drive/MyDrive/182-cv-project/data/tiny-imagenet-c.tar
!unzip drive/MyDrive/182-cv-project/data/imagenet-a.zip
!unzip drive/MyDrive/182-cv-project/data/imagenet-o.zip
# !tar -xvf drive/MyDrive/182-cv-project/data/imagenet-p_validation.tar

In [6]:
data_dir = pathlib.Path("tiny-imagenet-200")
imagenet_a_path = pathlib.Path('imagenet-a')
imagenet_o_path = pathlib.Path('imagenet-o')
tiny_imagenet_c_path = pathlib.Path('Tiny-ImageNet-C')
# imagenet_p_paths= ["shear", "spatter", "speckle_noise"]

In [7]:
#image_count = len(list(data_dir.glob('**/*.JPEG')))
!rm tiny-imagenet-200/.DS_Store
!rm tiny-imagenet-200/train/.DS_Store
!rm tiny-imagenet-200/val/.DS_Store
!rm tiny-imagenet-200/test/.DS_Store

rm: cannot remove 'tiny-imagenet-200/.DS_Store': No such file or directory
rm: cannot remove 'tiny-imagenet-200/train/.DS_Store': No such file or directory
rm: cannot remove 'tiny-imagenet-200/val/.DS_Store': No such file or directory
rm: cannot remove 'tiny-imagenet-200/test/.DS_Store': No such file or directory


In [8]:
images = pd.read_csv("tiny-imagenet-200/words.txt", names = ['Id', 'labels'], sep = '\t')

In [9]:
CLASS_NAMES = np.array([item.name for item in (data_dir / 'train').glob('*')])
CLASS_NAMES.sort()
num_classes = len(CLASS_NAMES)
print(len(CLASS_NAMES)) #should be 200
sum([cls in images['Id'].unique() for cls in CLASS_NAMES])

200


200

###Create new Validation Set using speckle/salt-pepper noise

In [46]:
def noisy(noise_typ,image):
   if noise_typ == "poisson":
      vals = len(np.unique(image))
      vals = 2 ** np.ceil(np.log2(vals))
      noisy = np.random.poisson(image * vals) / float(vals)
      return noisy.astype('uint8')
   elif noise_typ =="speckle":
      row,col,ch = image.shape
      gauss = np.random.randn(row,col,ch)
      gauss = gauss.reshape(row,col,ch)        
      noisy = image + image * gauss
      return noisy.astype('uint8')
   elif noise_typ == "s&p":
      row,col,ch = image.shape
      s_vs_p = 0.5
      amount = 0.004
      out = np.copy(image)
      # Salt mode
      num_salt = np.ceil(amount * image.size * s_vs_p)
      coords = [np.random.randint(0, i - 1, int(num_salt))
              for i in image.shape]
      out[coords] = 1

      # Pepper mode
      num_pepper = np.ceil(amount* image.size * (1. - s_vs_p))
      coords = [np.random.randint(0, i - 1, int(num_pepper))
              for i in image.shape]
      out[coords] = 0
      return out.astype('uint8')

def augmented_data(img):
    #input is an image path
    #returns list of images with following noise types applied: 
    #gauss, salt and pepper, poisson, speckle

    img = Image.open(img).convert('RGB')
    img = np.array(img)
    augmented = []

    noise_types = ["s&p", "poisson", "speckle"]
    for noise_type in noise_types: 
      ret_val = noisy(noise_type, img)
      
      augmented.append(ret_val)

    return augmented

In [48]:
# os.rmdir("speckle_augmented_val")
# os.mkdir("speckle_augmented_val")

prev_annotations = pd.read_csv("tiny-imagenet-200/val/val_annotations.txt", 
                              sep='\t', names=["Filename", "Class", "BB1", 
                                               "BB2", "BB3", "BB4"])
prev_annotations.drop(["BB1", "BB2", "BB3", "BB4"], axis=1, inplace=True)
new_annotations = pd.DataFrame(columns=["Filename", "Class"])

for img_file in os.listdir("tiny-imagenet-200/val/images"):
  path = "tiny-imagenet-200/val/images" + "/" + img_file
  augmented_imgs = augmented_data(path)
  for i, a in enumerate(augmented_imgs):
    im = Image.fromarray(a)
    prefix = img_file.split(".")[0]
    im.save("speckle_augmented_val/" + prefix + "_" + str(i) + 
            ".JPEG")
    new_annotations = new_annotations.append({"Filename" : prefix + "_" + str(i) + ".JPEG", 
                            "Class" : prev_annotations.loc[
                              prev_annotations["Filename"] == prefix + ".JPEG",
                              "Class"
                            ].item()}, ignore_index=True)
new_annotations.to_csv("speckle_augmented_val/augmented_val_annotations.txt", 
                       sep='\t', index=False, header=False)

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:22: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:28: FutureWarning: Using a non-tuple sequence for multidimensional indexing is deprecated; use `arr[tuple(seq)]` instead of `arr[seq]`. In the future this will be interpreted as an array index, `arr[np.array(seq)]`, which will result either in an error or a different result.


In [ ]:
!zip -r speckle_augmented_val.zip speckle_augmented_val/ 

In [54]:
new_annotations.to_csv("speckle_val_annotations.txt", 
                       sep='\t', index=False, header=False)

In [49]:

for item in imagenet_a_path.iterdir():
  if item.is_dir() and not item.name in CLASS_NAMES:
    shutil.rmtree(item)
for item in imagenet_o_path.iterdir():
  if item.is_dir() and not item.name in CLASS_NAMES:
    shutil.rmtree(item)
for name in CLASS_NAMES:
  if not pathlib.Path(os.path.join(imagenet_a_path, name)).is_dir():
    os.mkdir(os.path.join(imagenet_a_path, name))
  if not pathlib.Path(os.path.join(imagenet_o_path, name)).is_dir():
    os.mkdir(os.path.join(imagenet_o_path, name))

## Prediction Function

In [12]:
def predict(dataloaders, model): 
    """
    Run a forward pass (without caching data) for given model and return accuracy
    """
    if torch.cuda.is_available():
      model = model.cuda()

    accuracies = []
    model.eval()
    
    for phase in dataloaders.keys(): 
        counter = 0
        running_corrects = 0
        running_total = 0

        for inputs, labels in dataloaders[phase]: 
            inputs = inputs.to(device)
            labels.data = labels.data.to(device)
            outputs = model(inputs)
            _, preds = torch.max(outputs, 1)
            counter += 1
            
            running_corrects += torch.sum(preds == labels.data)
            running_total += len(preds)
            
        phase_acc = running_corrects / running_total
        accuracies.append(phase_acc)

    return accuracies
            

## Dataloaders

In [14]:
input_size = 299
batch_size = 32

In [15]:
data_transform = transforms.Compose([
        transforms.Resize((input_size, input_size)),
        transforms.ToTensor(),
        transforms.Normalize([0.485, 0.456, 0.406], [0.229, 0.224, 0.225])
    ])

In [17]:
image_datasets = {
    'imagenet-a': datasets.ImageFolder(imagenet_a_path, data_transform),
    'imagenet-o': datasets.ImageFolder(imagenet_o_path, data_transform),
}
for dir in tiny_imagenet_c_path.iterdir():
  image_datasets["imagenet-c-"+dir.name+"-2"] = datasets.ImageFolder(os.path.join(dir, "2"), data_transform)
# Create training and validation dataloaders

for dir in imagenet_p_paths: 
  path = pathlib.Path(dir)
  image_datasets["imagenet-p"+"-"+dir] = datasets.ImageFolder(path, data_transform)

dataloaders_dict = {x: torch.utils.data.DataLoader(image_datasets[x], 
                                                   batch_size=batch_size, 
                                                   shuffle=True, num_workers=2) 
                                                   for x in image_datasets.keys()}

RuntimeError: ignored

## Test All Models

In [ ]:
inception_model = torch.hub.load('pytorch/vision:v0.9.0', "inception_v3", pretrained=True)
inception_aux_in_ftrs = inception_model.AuxLogits.fc.in_features
inception_model.AuxLogits.fc = nn.Linear(inception_aux_in_ftrs, num_classes)
inception_in_ftrs = inception_model.fc.in_features
inception_model.fc = nn.Linear(inception_in_ftrs, num_classes)
resnet_model = torch.hub.load('pytorch/vision:v0.9.0', "resnet50", pretrained=True)
resnet_in_ftrs = resnet_model.fc.in_features
resnet_model.fc = nn.Linear(resnet_in_ftrs, num_classes)

Downloading: "https://github.com/pytorch/vision/archive/v0.9.0.zip" to /root/.cache/torch/hub/v0.9.0.zip
Downloading: "https://download.pytorch.org/models/inception_v3_google-1a9a5a14.pth" to /root/.cache/torch/hub/checkpoints/inception_v3_google-1a9a5a14.pth


Using cache found in /root/.cache/torch/hub/pytorch_vision_v0.9.0
Downloading: "https://download.pytorch.org/models/resnet50-19c8e357.pth" to /root/.cache/torch/hub/checkpoints/resnet50-19c8e357.pth


In [ ]:
#models_folder = pathlib.Path("drive/MyDrive/augmented_10")
models_folder = pathlib.Path("models")

In [ ]:
for item in models_folder.iterdir():
  if item.name.endswith(".pt"):
    if "inception" in item.name:
      model = inception_model
    elif "resnet50" in item.name:
      model = resnet_model
    else:
      print("ARCHITECTURE NOT FOUND")
      break
    print(item)
    model.load_state_dict(torch.load(item))
    accuracies = predict(dataloaders_dict, model)
    for i, key in enumerate(image_datasets.keys()):
      print(key + " accuracy = ", accuracies[i].item())

models/fine-tuned-inception-adv-training-with-stability.pt
imagenet-a accuracy =  0.04091985151171684
imagenet-o accuracy =  0.5161290168762207
imagenet-c-zoom_blur-2 accuracy =  0.3158999979496002
imagenet-c-pixelate-2 accuracy =  0.4136999845504761
imagenet-c-snow-2 accuracy =  0.39739999175071716
imagenet-c-impulse_noise-2 accuracy =  0.32580000162124634
imagenet-c-motion_blur-2 accuracy =  0.34139999747276306
imagenet-c-fog-2 accuracy =  0.5142999887466431
imagenet-c-jpeg_compression-2 accuracy =  0.39229997992515564
imagenet-c-shot_noise-2 accuracy =  0.25689998269081116
imagenet-c-contrast-2 accuracy =  0.4177999794483185
imagenet-c-gaussian_noise-2 accuracy =  0.24369999766349792
imagenet-c-defocus_blur-2 accuracy =  0.33969998359680176
imagenet-c-elastic_transform-2 accuracy =  0.3804999887943268
imagenet-c-glass_blur-2 accuracy =  0.24560000002384186
imagenet-c-frost-2 accuracy =  0.5004000067710876
imagenet-c-brightness-2 accuracy =  0.612500011920929


In [ ]:
torch.cuda.empty_cache()